# OSDM API Demo

This notebooks demonstrates how to access and use the OSDM API.

In [26]:
!pip install pip install python-dotenv
!pip install requests

## Read Credentials from ~.env File

In [17]:
import os
from dotenv import load_dotenv

%load_ext dotenv #Jupyter magic command to load .env variables
%dotenv #Jupyter magic command to load .env variables

In [21]:
KEY_SECRET = os.getenv('KEY_SECRET')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')

## Get Token from OSDM API

In [36]:
import requests
import json

In [47]:
#POST /oauth/v2/token HTTP/1.1

payload = {
"Host": "api.osdm-demo-test.cloud.sqills.com",
"Authorization": "Basic " + KEY_SECRET,
"Content-Type": "application/json",
"Content-Length": 125,
"grant_type": "https://com.sqills.s3.oauth.agent",
"username": USERNAME,
"password": PASSWORD
}

In [48]:
token = requests.post('https://api.osdm-demo-test.cloud.sqills.com/oauth/v2/token', data = payload)

In [50]:
token

<Response [503]>